In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adwaidmanoj","key":"f88b218e30b319687c2c15ba2a833bbf"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [3]:
!kaggle datasets list -s skin-disease


ref                                                                title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
subirbiswas19/skin-disease-dataset                                 Skin-Disease-Dataset                                  18126678  2023-10-14 18:13:04.083000           6274         78  0.75             
pacificrm/skindiseasedataset                                       Skin Disease Dataset                                1460921116  2024-11-23 11:11:23.487000           3034         33  1.0              
saurabhshahane/lumpy-skin-disease-dataset                          Lumpy Skin Disease Dataset                              697143  2022-11-10 14:32:48.750000           3233         62  0.8

In [4]:
!kaggle datasets download -d subirbiswas19/skin-disease-dataset


Dataset URL: https://www.kaggle.com/datasets/subirbiswas19/skin-disease-dataset
License(s): CC0-1.0
  0% 0.00/17.3M [00:00<?, ?B/s]
100% 17.3M/17.3M [00:00<00:00, 1.14GB/s]


In [5]:
!unzip skin-disease-dataset.zip -d skin_disease_data


Archive:  skin-disease-dataset.zip
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (1).webp  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (104).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (106).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (115).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (119).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (124).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (135).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (139).jpg  
  inflating: skin_disease_data/skin-disease-datasaet/test_set/BA- cellulitis/BA- cellulitis (144).jpg  
  inflating: skin_disease_data

In [10]:
import os

os.listdir("skin_disease_data")

['skin-disease-datasaet']

In [11]:
import os
os.listdir("skin_disease_data/skin-disease-datasaet/train_set")


['BA- cellulitis',
 'FU-ringworm',
 'VI-chickenpox',
 'BA-impetigo',
 'PA-cutaneous-larva-migrans',
 'FU-athlete-foot',
 'VI-shingles',
 'FU-nail-fungus']

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale = 1./255)

In [2]:
train_data = datagen.flow_from_directory(
    "/content/skin_disease_data/skin-disease-datasaet/train_set",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)


test_data = datagen.flow_from_directory(
     "/content/skin_disease_data/skin-disease-datasaet/test_set",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 924 images belonging to 8 classes.
Found 233 images belonging to 8 classes.


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models

In [9]:
base_model = MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224,224,3)
)
base_model_trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes,activation="softmax")
])

In [15]:
model.compile(
    optimizer='adam',
    loss = "categorical_crossentropy",
    metrics = ['accuracy']
)

In [18]:
history = model.fit(train_data,validation_data =test_data,epochs = 10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 252s 7s/step - accuracy: 0.6669 - loss: 1.0521 - val_accuracy: 0.3863 - val_loss: 6.7382
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 218s 8s/step - accuracy: 0.8782 - loss: 0.3820 - val_accuracy: 0.2961 - val_loss: 9.3556
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 211s 7s/step - accuracy: 0.9472 - loss: 0.1758 - val_accuracy: 0.1803 - val_loss: 12.0594
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 207s 7s/step - accuracy: 0.9410 - loss: 0.2436 - val_accuracy: 0.2704 - val_loss: 10.5199
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 216s 7s/step - accuracy: 0.9380 - loss: 0.2476 - val_accuracy: 0.2232 - val_loss: 19.3224
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 206s 7s/step - accuracy: 0.9591 - loss: 0.1486 - val_accuracy: 0.0987 - val_loss: 16.4454
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 206s 7s/step - accuracy: 0.9484 - loss: 0.2530 - val_accuracy: 0.2361 - val_loss: 11.1329
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 216s 7s/step - accuracy: 0.9509 - loss: 0.1942 - val_accuracy: 0.141

In [19]:
model.save("skin_disease_model.h5")

In [20]:
from google.colab import files
files.download("skin_disease_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>